# Building a docker container for deploying, hosting and monitoring our LightGBM model

Now that we have successfully developed the LightGBM model for training, we can proceed and finish our custom Docker container for **serving the model and provide inference logic**. We'll use the [sagemaker-inference-toolkit](https://github.com/aws/sagemaker-inference-toolkit) library to facilitate this tasks.

This part of the workshop is composed of 4 parts:
1. <a href="#custom_inference_container">Extend our previous <strong>custom Docker container for serving the model</strong> with SageMaker</a>
2. <a href="#inference_toolkit">Create a Python package to configure <strong>SageMaker Inference toolkit</strong></a>
3. <a href="#container_inference_build"><strong>Build our final Docker image for training and inferece</strong> and <strong>push</strong> it to Amazon Elastic Container Registry</a>
3. <a href="#testing_inference"><strong>Testing the inference locally</strong> with our container using the SageMaker Python SDK</a>

<div id=custom_inference_container>
<h2> 1. Creating the inference container</h2>
</div>
Again, we start by defining some variables like the current execution role, the ECR repository that we are going to use for pushing the final custom Docker container and the default Amazon S3 bucket to be used by Amazon SageMaker:

In [ ]:
import boto3
import sagemaker
from sagemaker import get_execution_role

ecr_repository_name = 'sagemaker-custom-lightgbm'
role = get_execution_role()
account_id = role.split(':')[4]
region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
bucket = sagemaker_session.default_bucket()

print('ecr_repository_name:', ecr_repository_name)
print('account_id:',account_id)
print('region:',region)
print('role:',role)
print('bucket:',bucket)

This time we'll write all the Python modules from this notebook with the `%%writefile` built-in magic commands from IPython.

We'll store our code in a few directories:
```
1_custom_inference/
│ 
├── docker
│   └── code
│   
└── package
    ├── src
    └── custom_lightgbm_inference
```

In [ ]:
!rm -rf ../package
!rm -rf ../docker

!mkdir ../docker
!mkdir ../docker/code

!mkdir ../package
!mkdir ../package/src
!mkdir ../package/src/custom_lightgbm_inference
!touch ../package/src/custom_lightgbm_inference/__init__.py

Let's write another Dockerfile for building our custom end-to-end LightGBM container for **training and serving**.

We'll use the previous container already with the [SageMaker Training Toolkit](https://github.com/aws/sagemaker-training-toolkit) and extend it with [SageMaker Inference Toolkit](https://github.com/aws/sagemaker-inference-toolkit) and with our inference logic.

**By serving** you can understand: exposing our model as a webservice that can be called through an API call.

In [ ]:
%%writefile ../docker/Dockerfile
# 1. Use with our previous container as base image
FROM sagemaker-custom-lightgbm:latest

# 2. Defining some variables used at build time to install Python3
ARG PYTHON=python3
ARG PYTHON_PIP=python3-pip
ARG PIP=pip3
ARG PYTHON_VERSION=3.6.6

# 3. Set a docker label to advertise multi-model support on the container 
LABEL com.amazonaws.sagemaker.capabilities.multi-models=false
# Set a docker label to enable container to use SAGEMAKER_BIND_TO_PORT environment variable if present
LABEL com.amazonaws.sagemaker.capabilities.accept-bind-to-port=true

# 4. Instal libraries for the sagemaker-inference and multi-model-server libraries
RUN apt-get update -y && apt-get -y install --no-install-recommends default-jdk
RUN rm -rf /var/lib/apt/lists/*

# 5. Copy our package to the WORKDIR
COPY code/custom_lightgbm_inference-0.1.0.tar.gz /custom_lightgbm_inference-0.1.0.tar.gz
        
# 6. Installing our custom package for inference
RUN ${PIP} install --no-cache --upgrade \
        /custom_lightgbm_inference-0.1.0.tar.gz && \
    rm /custom_lightgbm_inference-0.1.0.tar.gz

# 7. Set SageMaker serving environment variables
ENV SM_MODEL_DIR /opt/ml/model
ENV CODE_DIR /opt/ml/code

> This time, for simplicity, we won't provide logic to be injected inside the container for inference. We'll stick a a pre-defined default inference logic (we don't set environment variables like `ENV SAGEMAKER_SERVING_MODULE sagemaker_custom.serving:main`). However, if desired, we could do something similar to [what was shown in the training lab](../../0_custom_train/lab/1_training-container.ipynb) to provide user-defined logic for inference.
>
> SageMaker will run our Docker image for training with `docker run <YOUR-IMAGE> serve` ([more details here](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-inference-code.html)). After installing our custom Python package we can run it as a command line script when executing `serve` in the terminal.

<div id="inference_toolkit">
    <h2>2. Creating our Python Package with the SageMaker Inference Toolkit</h2>
</div>

The **Inference Handler** is how we use the SageMaker Inference Toolkit to encapsulate our code and expose it as a SageMaker container. We create a `handler.py` module:

In [ ]:
%%writefile ../package/src/custom_lightgbm_inference/handler.py
import os
import sys
import joblib
from sagemaker_inference.default_inference_handler import DefaultInferenceHandler
from sagemaker_inference.default_handler_service import DefaultHandlerService
from sagemaker_inference import content_types, errors, transformer, encoder, decoder

class HandlerService(DefaultHandlerService, DefaultInferenceHandler):
    def __init__(self):
        op = transformer.Transformer(default_inference_handler=self)
        super(HandlerService, self).__init__(transformer=op)
    
    ## Loads the model from the disk
    def default_model_fn(self, model_dir):
        model_filename = os.path.join(model_dir, "model.joblib")
        return joblib.load(model_filename)
    
    ## Parse and check the format of the input data
    def default_input_fn(self, input_data, content_type):
        if content_type != "text/csv":
            raise Exception("Invalid content-type: %s" % content_type)
        return decoder.decode(input_data, content_type).reshape(1,-1)
    
    ## Run our model and do the prediction
    def default_predict_fn(self, payload, model):
        return model.predict( payload ).tolist()
    
    ## Gets the prediction output and format it to be returned to the user
    def default_output_fn(self, prediction, accept):
        if accept != "text/csv":
            raise Exception("Invalid accept: %s" % accept)
        return encoder.encode(prediction, accept)

Now we need to create the entrypoint of our Python package. The `main()` function will be ran as a console script later:

In [ ]:
%%writefile ../package/src/custom_lightgbm_inference/my_serving.py
from sagemaker_inference import model_server
from custom_lightgbm_inference import handler

HANDLER_SERVICE = handler.__name__

def main():
    print('Running handler service:', HANDLER_SERVICE)
    model_server.start_model_server(handler_service=HANDLER_SERVICE)


Finally we define our `setup.py` file to create our custom inference package with setuptools.

We setup a entry point so that by running `serve` in the terminal our model server will start (we run the main function above).

In [ ]:
%%writefile ../package/setup.py
from __future__ import absolute_import

from glob import glob
import os
from os.path import basename
from os.path import splitext

from setuptools import find_packages, setup

setup(
    name='custom_lightgbm_inference',
    version='0.1.0',
    description='Custom container serving package for SageMaker.',
    keywords="custom container serving package SageMaker",

    packages=find_packages(where='src'),
    package_dir={'': 'src'},
    py_modules=[splitext(basename(path))[0] for path in glob('src/*.py')],
    
    install_requires=[
        'sagemaker-inference==1.3.0',
        'multi-model-server==1.1.2'
    ],
    entry_points={"console_scripts": ["serve=custom_lightgbm_inference.my_serving:main"]},
)


<div id="container_inference_build">
<h2>3. Build and push the container</h2>
</div>

We are now ready to build this container and push it to Amazon ECR. This task is executed using a shell script stored in the `../script/` folder like before. Let's take a look at this script and then execute it.

In [ ]:
! pygmentize ../scripts/build_and_push.sh

---
**Let's run this script now:**

In [ ]:
print('Confgurations for our bash script:')
print('account ID:', account_id)
print('region:', region)
print('ECR repository name:', ecr_repository_name)

In [ ]:
! ../scripts/build_and_push.sh $account_id $region $ecr_repository_name

[Go to ECR in the AWS console](https://console.aws.amazon.com/ecr/home?region=us-east-2) and check if our `sagemaker-custom-lightgbm` repository has the updated Docker image.

![ecr-repo-updated](./media/ecr-repo-updated.png)

---

<div id="testing_inference">
<h2>4. End-to-end test with Amazon SageMaker</h2>
</div>

Once we have correctly pushed our container to Amazon ECR, we are now ready for our final test using with Amazon SageMaker. 

As previously explained, we have 2 options use our LightGBM Docker image: via **Script Mode** (using the `sagemaker.estimator.Estimator` class) or **Framework mode** (using the `sagemaker.estimator.Framework` class).

For simplicity, we will use the **Script Mode** (to use the Framework mode we would have to implement the [Framework Model class](https://sagemaker.readthedocs.io/en/stable/api/inference/model.html#sagemaker.model.FrameworkModel)).

*With  the SageMaker Python SDK we will:*

a) **train a LightGBM model** using the [sagemaker.estimator.Estimator](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.Estimator) class and calling the `fit(...)` method

b) **deploy our LightGBM model** calling the `deploy(...)` method of the estimator and receiving a [Predictor object](https://sagemaker.readthedocs.io/en/stable/api/inference/predictors.html#sagemaker.predictor.Predictor)

c) **send requests to our deployed LightGBM model** calling the `predict(...)` method of the predictor object
    

In [ ]:
container_image_uri = '{0}.dkr.ecr.{1}.amazonaws.com/{2}:latest'.format(account_id, region, ecr_repository_name)
print(container_image_uri)

In [ ]:
train_config = f's3://sagemaker-{region}-{account_id}/sagemaker-custom/data/iris_train.csv'
test_config = f's3://sagemaker-{region}-{account_id}/sagemaker-custom/data/iris_test.csv'

In [ ]:
sources = f's3://sagemaker-{region}-{account_id}/sagemaker-custom/code/sourcedir.tar.gz'

### a) train a LightGBM model

In [ ]:
import sagemaker
import json

# JSON encode hyperparameters.
def json_encode_hyperparameters(hyperparameters):
    return {str(k): json.dumps(v) for (k, v) in hyperparameters.items()}

hyperparameters = json_encode_hyperparameters({
    "sagemaker_program": "train.py",
    "sagemaker_submit_directory": sources,
    "num_leaves": 40,
    "max_depth": 10,
    "learning_rate": 0.11,
    "random_state": 42})

prefix = 'sagemaker-custom-final'

estimator = sagemaker.estimator.Estimator(container_image_uri,
                                    role,
                                    train_instance_count=1, 
                                    train_instance_type='local',
                                    #train_instance_type='ml.m5.xlarge',
                                    base_job_name=prefix,
                                    hyperparameters=hyperparameters)

In [ ]:
estimator.fit({'train': train_config, 'validation': test_config })

### b) deploy our LightGBM model

In [ ]:
predictor = estimator.deploy(initial_instance_count=1,
                 instance_type='local',
                )

### c) send requests to our deployed LightGBM model 

In [ ]:
import pandas as pd
import random
from sagemaker.predictor import csv_serializer, csv_deserializer

# configure the predictor to do everything for us
predictor.content_type = 'text/csv'
predictor.accept = 'text/csv'
predictor.serializer = csv_serializer
predictor.deserializer = None

# load the testing data from the validation csv
validation = pd.read_csv('../../0_custom_train/lab/data/test/iris_test.csv', header=None)
idx = random.randint(0,len(validation)-5)
req = validation.iloc[idx:idx+5].values

# cut a sample with 5 lines from our dataset and then split the label from the features.
X = req[:,0:-1].tolist()
y = req[:,-1].tolist()

In [ ]:
# call the local endpoint
for features,label in zip(X,y):
    prediction = float(predictor.predict(features).decode('utf-8').strip())

    # compare the results
    print("\nRESULT: {} == {} ? {}\n".format( label, prediction, label == prediction ) )

---
## The end of the custom Docker container development! 

If you'd like to dive even deeper into how to create your own custom Docker containers for SageMaker, a nice place to start is looking at the **open source implementations of a few SageMaker containers**:

- **SageMaker Scikit-Learn container**: https://github.com/aws/sagemaker-scikit-learn-container

- **SageMaker XGBoost container**: https://github.com/aws/sagemaker-xgboost-container



Note that these SageMaker containers use similar similar logic with Python package console scripts and similar libraries to what we used here:
- multi-model-server
- sagemaker-inference
- sagemaker-training

(Take a look at the [setup.py](https://github.com/aws/sagemaker-scikit-learn-container/blob/master/setup.py) and [requirement.txt](https://github.com/aws/sagemaker-scikit-learn-container/blob/master/requirements.txt) files for more details).
    
---

## What's next?

### Let's automate everything in a ML pipeline!

## &rarr; [CLICK HERE TO MOVE ON](../../2_pipeline/3_ml-pipeline.ipynb)